<a href="https://colab.research.google.com/github/adsamardeep/Identify-the-dance-form/blob/master/hackathons/hackerearth_dancer3_vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np 
import pandas as pd 
import cv2, pickle
from PIL import Image 
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.applications.vgg19 import VGG19
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
IMG_WIDTH=224
IMG_HEIGHT=224
BATCH_SIZE = 15

In [0]:
train = pickle.load(open("/content/drive/My Drive/HackerEarth/IdentifyDanceForms/train.npy",'rb'))
test = pickle.load(open("/content/drive/My Drive/HackerEarth/IdentifyDanceForms/test.npy",'rb'))
train = np.array(train)
test = np.array(test)
train_data = pd.read_csv("/content/drive/My Drive/HackerEarth/IdentifyDanceForms/train.csv")

encoder = LabelEncoder()
encoder.fit(train_data["target"])
encoded_Y = encoder.transform(train_data["target"])
y_labels = np_utils.to_categorical(encoded_Y)

In [0]:

from tensorflow.keras.models import Model
import tensorflow.keras as keras

resnet = VGG19(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3),pooling='max')

output = resnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
resnet = Model(resnet.input, output)

In [0]:
for layer in resnet.layers:
    layer.trainable = False

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

num_classes = 8

model = Sequential()
model.add(resnet)
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 512)               20024384  
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 8200      
Total params: 20,557,896
Trainable params: 533,512
Non-trainable params: 20,024,384
_________________________________________________________________


In [0]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                              restore_best_weights=False
                                              )


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.3,
                                   patience=5,
                                   verbose=1,
                                   min_delta=1e-3,min_lr = 1e-7,
                                   )

In [0]:
model.compile(optimizer = adam, 
              loss = 'categorical_crossentropy', 
              metrics=['accuracy',tfa.metrics.F1Score(num_classes=num_classes)])

In [21]:
train.shape

(364, 224, 224, 3)

In [22]:
y_labels.shape

(364, 8)

In [23]:
model.fit(train, y_labels,steps_per_epoch=np.ceil(float(train.shape[0]) / float(BATCH_SIZE)),
                        epochs = 50,callbacks=[early_stop,reduce_lr])

Epoch 1/50
25/25 [==============================] - 4s 174ms/step - loss: 64.6023 - accuracy: 0.1319 - f1_score: 0.1291 - lr: 1.0000e-04
Epoch 2/50
25/25 [==============================] - 4s 174ms/step - loss: 39.9133 - accuracy: 0.2500 - f1_score: 0.2396 - lr: 1.0000e-04
Epoch 3/50
25/25 [==============================] - 4s 174ms/step - loss: 29.7655 - accuracy: 0.3407 - f1_score: 0.3435 - lr: 1.0000e-04
Epoch 4/50
25/25 [==============================] - 4s 174ms/step - loss: 21.3636 - accuracy: 0.4505 - f1_score: 0.4475 - lr: 1.0000e-04
Epoch 5/50
25/25 [==============================] - 4s 174ms/step - loss: 18.5862 - accuracy: 0.5082 - f1_score: 0.5031 - lr: 1.0000e-04
Epoch 6/50
25/25 [==============================] - 4s 173ms/step - loss: 16.6555 - accuracy: 0.5577 - f1_score: 0.5584 - lr: 1.0000e-04
Epoch 7/50
25/25 [==============================] - 4s 174ms/step - loss: 12.7167 - accuracy: 0.5797 - f1_score: 0.5803 - lr: 1.0000e-04
Epoch 8/50
25/25 [=======================

In [0]:
pred = model.predict(test)
pred = np.argmax(pred, axis=1)
test_data = pd.read_csv("/content/drive/My Drive/HackerEarth/IdentifyDanceForms/test.csv")

pred = encoder.inverse_transform(pred)
result = pd.DataFrame(pred, test_data["Image"], columns=["target"])
result.to_csv("sample5.csv")